# Project - Prototyping and Communicating Results in Business Analytics

In business, prototyping involves making an approximate analysis with some lax assumptions to evaluate a business proposition, instead of making a full-fledged analysis.
In business environments, there's a large swath of people who we may need to communicate with. They will have varying degrees of data literacy, which makes it harder to not only report results, but also:

- Makes it harder for these colleagues to request things of us
- Makes it harder for us to manage a large and varied influx of requests

In this project, we'll build a prototype and  communicate the results effectively as a data professional with people whose data skills aren't as sharp as our own.

### Scenario

We're working for one of Google's data science teams and someone from another team, specifically an account manager — "a person who works for a company and is responsible for the management of sales and relationships with particular customers" — approaches you with a request. She wants to take a prophylactic approach and improve the revenue of undervalued apps to motivate the developers to keep working on them. 

Since her department's budget for marketing won't allow her to invest on ads (which would boost the number of sales), the only way to improve the revenue is by tweaking the price. She requests that we determine which paid apps are undervalued (undervalued here means that their price could be increased without lowering demand).

We talk it over with our manager and ou come to the decision of **prototyping** this project, that is, making a rough analysis with some lax assumptions, instead of making a full-fledged analysis. Prototyping has several advantages:

- Having a prototype makes it much easier to estimate the cost of a fully fleshed-out project, be it human, technological, or financial resources.
- Having a more accurate estimate of the cost allows decision-makers to not go through with the project if it doesn't seem profitable enough, or not the best allocation of resources at that point in time.
- It allows grunt workers (like ourselves) to change and add missing requirements to complete the project. Oftentimes these are overlooked and only spotted later; some examples are missing data, the creation of a new database, the development of an API.
- It allows business people (like the account manager) to tweak the project's goal.
- It gives all stakeholders an opportunity to add easily accomplishable side-goals given the main goal, thus maximizing the project's output.

Despite the above, we should not only limit the scope of the project (focus on one goal) when prototyping, but we should also timebox it — prototypes without deadlines run the danger of becoming time wasters that do not let us capitalize on the above advantages.

**Dataset** We'll be using The Google Playstore apps`googleplaystore.csv` dataset available at [kaggle](https://www.kaggle.com/lava18/google-play-store-apps) and leverage the power of pandas!  

In [ ]:
# Data Exploration

import pandas as pd
playstore = pd.read_csv("googleplaystore.csv")
print(playstore.shape)

In [ ]:
print(playstore.info())

In [ ]:
print(playstore.head(5))

### Cleaning Data

First, we will identify null,missing and duplicate entries and then devise a strategy to deal with them

In [ ]:
null_content = playstore[playstore["Content Rating"].isnull()]
print(null_content) # NaN in "Content Rating Column" of index 10472
print('\n', null_content["Content Rating"])

In [ ]:
null_type = playstore[playstore["Type"].isnull()]
print(null_type) # NaN in "Type Column" of index 9148

print('\n', null_type["Type"])

In [ ]:
null_c_ver = playstore[playstore["Current Ver"].isnull()]
print(null_c_ver) # NaN in "Current Ver Column" of 8 indices

print('\n', null_c_ver["Current Ver"])

In [ ]:
null_a_ver = playstore[playstore["Android Ver"].isnull()]
print(null_a_ver) # NaN in "Android Ver Column" of 3 indices

print('\n', null_a_ver["Android Ver"])

In [ ]:
# Duplicate Entries
unique_apps = pd.Series(playstore["App"].unique())

print(unique_apps.shape)
duplicates = playstore["App"].duplicated(keep='last') 
print(playstore[duplicates].shape)
print(type(unique_apps))

### Other Data Cleaning Issues
Apart from missing, null and duplicate entries as above, there are a few other data cleaning issues requiring our attention:

**Data Type**
- The columns `Reviews`, `Installs`, `Price` and `Size` have to be `float` or `int` types for analysis, presently they are `object` types.

**Free Apps**
- Our prototyping is inclined towards determining the "under-valued" apps for improving revenues, so "Free apps" are superfluous for our analysis.

### Data Cleaning Strategy
Our **Data Cleaning Strategy** is based on following:
- Some of the data type issues are due to a particularly problematic entry at `index 10472` which has a missing entry for the column `Category` resulting in erratic data layout; so, we will remove it first. It's a free app, so its removal will make no difference for overall analysis.

- Second, we will remove the "Free Apps" followed by removal of `Type` column to focus on relevant data only.

- Third, we will correct the `Price` column through vectorized string methods and convert it to `float` type.

- Fourth, we will correct the `Size` column using Series.apply() method and defining a function for cleaning. Then, we will convert it to `int` type.

In [ ]:
# Removal of entry 10472
playstore.drop(labels=10472, inplace=True)
print(playstore.info())

In [ ]:
# Price Column - remove $ and convert to float
playstore["Price"] = playstore["Price"].str.replace("$", "").astype("float")


In [ ]:
# `Reviews` Column convert to int
playstore["Reviews"] = playstore["Reviews"].astype("int")

In [ ]:
# Installs column - remove "," and "+" and convert to int
playstore["Installs"] = playstore["Installs"].str.replace(",","").str.replace("+","").astype("int")



In [ ]:
# Size Column - remove "M" and "K", convert to float, express in uniform units i.e. MB

def clean_size(size):
    """Convert file size string to float and megabytes"""
    size = size.replace("M","")
    if size.endswith("k"):
        size = float(size[:-1])/1000
    elif size == "Varies with device":
        size = pd.np.NaN
    else:
        size = float(size)
    return size

playstore["Size"] = playstore["Size"].apply(clean_size)



In [ ]:
# Remvoing "Free" Apps and "Type" column
playstore = playstore[playstore["Type"] == "Paid"]
playstore.drop(labels="Type", axis=1, inplace=True)
print(playstore.info())

In [ ]:
# Taking care of duplicates

unique_apps = pd.Series(playstore["App"].unique())

print(unique_apps.shape)
duplicates = playstore["App"].duplicated(keep='first').sum() 
print(duplicates)

# We have to remove 44 duplicate apps
# We will remove duplicates whose number of reviews are not the highest
# First, we will sort by "Reviews" 
playstore.sort_values("Reviews", ascending=False, inplace=True)

playstore = playstore.drop_duplicates(subset="App", keep="first")

print(playstore["App"].duplicated(keep='first').sum())

In [ ]:
# Resetting Index
playstore.reset_index(inplace=True, drop=True)

print(playstore.info())

In [ ]:
print(playstore.head(5)) # index is reset

### Data Exploration
We are interested in identifying "under-valued apps". But, we are not yet clear about the meaning of this term. For a better understanding, we need to first explore the data starting from the `price` column.` 

In [ ]:
# Check price distribution

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

sns.set_style("white")
sns.set_context("poster")

sns.distplot(playstore["Price"], bins=20, kde=False, rug=True)
plt.show()

In [ ]:
sns.set_context("poster")

sns.kdeplot(playstore["Price"], shade=True)
plt.show()

In [ ]:
sns.set_context("paper")
sns.boxplot(data=playstore["Price"])

### Outliers
Above distribution shows, most apps are priced under $50 with some outliers at various prices up to $400. Let's first investigate outliers.   

In [ ]:
# Apply a Boolean mask of apps priced over $50
expensive = playstore[playstore["Price"]>50]

fig,ax=plt.subplots()
figsize=(12,6)
ax.hist(expensive['Price'],range=(50,400), bins=20)
plt.show()

In [ ]:
# Apply a Boolean mask of apps priced under $50
affordable_apps = playstore[playstore["Price"]<=50]

fig,ax=plt.subplots()
figsize=(12,6)
ax.hist(affordable_apps['Price'],range=(0,50), bins=20)
plt.show()

 Even in the range of <$50, most applications seem to be costing $5 and below. So, we need to further refine our results 

In [ ]:
# Create a boolean mask for apps price less than $5 and one for $5 and above
cheap = affordable_apps[affordable_apps["Price"]<5]

reasonable = affordable_apps[affordable_apps["Price"]>=5]

cheap.hist(column="Price", grid=False, figsize=(12,6))

reasonable.hist(column="Price", grid=False, figsize=(12,6))

In [ ]:
# Adding a column "affordability" in affordable_apps as a price-metric

affordable_apps["affordability"] = affordable_apps["Price"].apply(lambda price: "cheap" if price<5 else "reasonable").copy()

In [ ]:
print(affordable_apps[["Price", "affordability"]].head())

## Data Analysis Strategy

Now, that we have created a dataframe of undervalued apps `affordable_apps` and segmented it further into `cheap` and `reasonable` from the point of view of `affordability`, we can focus our analysis on developing metrics which would evaluate the price with respect to `Ratings`, `Genre` and `Category`. This implies that a cheaper application will need to qualify on some of the other criteria in order to be considered for price increase. Similarly, a somewhat expensive application will not be automatically ruled out if it passes other criteria.

- We will first consider `Price` vs `Ratings` analytics. If no correlation exists, we will develop a `price_criterion` based on mean price.
- Next, we will develop a `genre_criterion` to see how apps prices fair against `Genre`
- Finally, a `category_criterion` will help us determine how prices are related to `Category`
- In the end. a majority vote of all criteria will decide whether the app is to be considered for price increase. 

### Price versus Rating
We can now compare `Price` with `Ratings` and other factors such as `Genre`, `Category`, `Reviews`, `Size`, `Installs` etc. 

In [ ]:
# Determine correlations
correlations = affordable_apps.corr()
print(correlations)

In [ ]:
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(correlations, annot=True, linewidths=.5, ax=ax)

We can see above that no significant correlations exist except between `Installs` and `Reviews` (0.81). Correlation between `Price` and `Rating` is non-existent (-0.05). This implies that we can rely on a metric like "mean_price" and consider increasing the prices of apps which lie below mean  price. 

In [ ]:
# Determine correlations
cheap = affordable_apps["Price"]<5
correlations_cheap = affordable_apps[cheap].corr()
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(correlations_cheap, annot=True, linewidths=.5, ax=ax)

In [ ]:
# Find mean of cheap apps
cheap = affordable_apps["Price"]<5
cheap_mean = affordable_apps.loc[cheap, "Price"].mean()
print(cheap_mean)

# For only the cheap apps, create a column in affordable_apps called price_criterion 
# that takes the value 1 when the app's price is lower than cheap_mean, and 0 otherwise.

affordable_apps.loc[cheap, "price_criterion"] = affordable_apps["Price"].apply(lambda x: 1 if x<cheap_mean else 0)

affordable_apps[cheap].plot(kind="scatter", x="Price", y="Rating")

In [ ]:
# Determine correlations
reasonable = affordable_apps["Price"]>=5
correlations_reasonable = affordable_apps[reasonable].corr()
f, ax = plt.subplots(figsize=(9, 6))
sns.heatmap(correlations_reasonable, annot=True, linewidths=.5, ax=ax)

In [ ]:
# Find mean of reasonable apps
reasonable = affordable_apps["Price"]>=5

reasonable_mean = affordable_apps.loc[reasonable, "Price"].mean()
print(reasonable_mean)

# For only the cheap apps, create a column in affordable_apps called price_criterion 
# that takes the value 1 when the app's price is lower than cheap_mean, and 0 otherwise.

affordable_apps.loc[reasonable, "price_criterion"] = affordable_apps["Price"].apply(lambda x: 1 if x<cheap_mean else 0)

affordable_apps[reasonable].plot(kind="scatter", x="Price", y="Rating")

### Price versus Genres

Looking at the first few rows of affordable_apps, we see that for an app belonging to multiple genres, the `Genre` entries are separated by `;`, so `groupby ("Genres")` would not yield meaningful results as there are not many `unique` entries. We will instead resort to counting genres to which a genre belongs and then develop a metric for comparing with mean price.

In [ ]:
# Genre Count based on separator ';'
# str.count() takes a regex
affordable_apps["genre_count"] = affordable_apps["Genres"].str.count(';')+1 


print(affordable_apps["genre_count"].value_counts())


Above, most apps belong to one genre and few to 2 genres. Let's see, how their mean price varies based on genre_count. 

In [ ]:
genres_mean = affordable_apps.groupby(["affordability", "genre_count"]).mean()[["Price"]]


print(genres_mean)

In [ ]:
# Alternately 
genres_mean = affordable_apps.groupby(["affordability", "genre_count"])["Price"].mean()
genres_mean = pd.DataFrame(genres_mean) # we convert the groupby object to Dataframe for ease of reference
print(genres_mean.info()) # We get a multi-index dataframe with one data column "Price"
print(genres_mean)


In [ ]:
# We can check genre_count against other criteria as well
all_mean = affordable_apps.groupby(["affordability", "genre_count"]).mean()


print(all_mean)

**`genre_criterion` column** 
Now, we develop a criteria for `Genres` verus `Price`. Let's call it `genre_criterion`. We define a function which will compare app `Price` against `genres_mean` and assign it a value of 1 if less than `genre_mean` and 0 if greater or equal to `genres_mean`

In [ ]:
def label_genres(row):
    """For each segment in `genres_mean`,
    labels the apps that cost less than its segment's mean with `1`
    and the others with `0`."""
    
    aff = row["affordability"]
    gc = row["genre_count"]
    price = row["Price"]
    
    if price < genres_mean.loc[(aff, gc)][0]:
        return 1
    else:
        return 0

affordable_apps["genre_criterion"] = affordable_apps.apply(label_genres, axis="columns")

print(affordable_apps[["Price","genre_count","affordability","genre_criterion"]].head())
    

### Price versus Category

`groupby ("Categories")` is a viable option looking at `value_counts` below. We will group according to categories and then develop a metric for comparing with mean price.

In [ ]:
# Check unique Category entries and their count

print(affordable_apps["Category"].value_counts())

In [ ]:
# Create a dataframe similar to genres_means called categories_mean

categories_mean = affordable_apps.groupby(["affordability", "Category"])["Price"].mean()
categories_mean = pd.DataFrame(categories_mean) # we convert the groupby object to Dataframe for ease of reference

print(categories_mean.info()) # We get a multi-index dataframe with one data column "Price"

print(categories_mean.head(5))

print(categories_mean.tail(5))

In [ ]:
# Apply the function analogous to label_genres and create column "category_criterion"

def label_cats(row):
    """For each segment in `categories_mean`,
    labels the apps that cost less than its segment's mean with `1`
    and the others with `0`."""
    
    aff = row["affordability"]
    cat = row["Category"]
    price = row["Price"]
    
    if price < categories_mean.loc[(aff, cat)][0]:
        return 1
    else:
        return 0

affordable_apps["category_criterion"] = affordable_apps.apply(label_cats, axis="columns")

print(affordable_apps[["Price","Category","affordability","category_criterion"]].head())

### Results and Impacts

We will use a technique called "majority voting", in which we decide whether an app's price should increase based on all criteria. For each app, we're going to count the number of votes for each result, and the majority will be declared the winner.

In [ ]:
# Create a column Result based on majority vote of all criteria

criteria = ["price_criterion", "genre_criterion", "category_criterion"]

affordable_apps["Result"] = affordable_apps[criteria].mode(axis=1)

print(affordable_apps.sample(5))

In [ ]:
# Number of apps voted for price increase and percentage

nr_eligible= affordable_apps["Result"].sum()

percent_eligible = (nr_eligible/affordable_apps["Result"].size)*100

print(nr_eligible, percent_eligible, sep='\n')

### Impact Analysis

- Since we do not have data for the sales, we will use `Installs` as a (rough) proxy for the number of times the app was bought.
- We will compare mean prices `cheap_mean` and `affordable_mean` with current `Price`, and if it is higher, we will adopt it as `New Price`.
- Impact will be assessed by multiplying difference between `New Price` and current `Price` with `Installs` and recorded as `Impact`. 

In [ ]:
# Calculating cheap_mean and reasonable_mean
cheap = affordable_apps["Price"]<5
cheap_mean = affordable_apps.loc[cheap, "Price"].mean()

reasonable = affordable_apps["Price"]>=5

reasonable_mean = affordable_apps.loc[reasonable, "Price"].mean()

In [ ]:
# New Price column
affordable_apps["New Price"] = affordable_apps["Price"].apply(lambda x: round(max(x,cheap_mean),2) if x<5 else round(max(x,reasonable_mean),2))
    
print(affordable_apps[["Price","New Price"]].sample(5))                                    
                                    

In [ ]:
# Alternately (New Price)
# criteria = ["price_criterion", "genre_criterion", "category_criterion"]
# affordable_apps["Result"] = affordable_apps[criteria].mode(axis='columns')
# def new_price(row):
#     if row["affordability"] == "cheap":
#         return round(max(row["Price"], cheap_mean), 2)
#     else:
#         return round(max(row["Price"], reasonable_mean), 2)
    
# affordable_apps["New Price"] = affordable_apps.apply(new_price, axis="columns")

In [ ]:
# Assessing Impact

affordable_apps["Impact"] = (affordable_apps["New Price"] - affordable_apps["Price"])* affordable_apps["Installs"]

print(affordable_apps[["Price","New Price","Installs","Impact"]].sample(5)) 

total_impact = affordable_apps["Impact"].sum()
print('\n', "Total Impact: ", round(total_impact/1000000, 2), " million")

### Analysis of Results
- Above results are not realistic. They tend to imply that a minor price tweaking in a few hundred apps can generate additional revenues of $115 million.
- The problem is obviously with the proxy `Installs` that we have used in place of future sales projection. 
- Our Prototype needs to be refined further in collaboration with the Sales and Marketing teams to analyze future sales projections if prices are increased by a certain factor.
- Nonetheless, the proposition has a potential if supported by marketing.

### Communicating Results - Summary

Below is a summary of our Prototyping effort.

**Data Quality**

We found one erroneous data point (10472) which not only wasn't important, it was easy to fix. We should monitor this if management decides to move forward with the project to make sure it doesn't become an issue later on.
We also found quite a few duplicates that were relatively easy to handle, but it would be better to deal with this from the start, lest it becomes a hard-to-manage problem.

**Apps Prices**

Since the main goal of this project is to find undervalued apps, price is the central theme through which our analysis will revolve around.

As a whole, the distribution of prices among the paid apps is tremendously skewed.Even when we removed the outliers from the analysis and focused on the apps costing less than $50, We ended up with a little over 700 rows of data, but the tendency persists.  

Due to this reason, we decided to separate the apps into two different groups. Those that cost less than $5 — from now on the cheap apps — and the remaining ones — from now on the reasonable apps_. We'll call this feature "Affordability".

**Strategy**

In order to limit the scope of this prototype, we included only four more features in our analysis: the ratings, the affordability, the categories, and the genres.

We separated the data into price groups and broke those down according to category and genre to determine the candidates for price increases.

**Prices vs. Other Features**

We've found there isn't any meaningful relationship between the price and the rating.

As an example, in the scatter plot , we have the price on the horizontal axis, and the rating on vertical axis. We see that price's column of data points are virtually indistinguishable from one another.

In [ ]:
affordable_apps[cheap].plot(kind="scatter", x="Price", y="Rating")

As for the genres, since each app can belong to multiple genres, and given that we only have around 700 rows of data, we decided not to drill down into the genres, but instead limit our analysis to counting the number of genres for each app and computing the average.

In [ ]:
genres_mean = affordable_apps.groupby(["affordability", "genre_count"])["Price"].mean()
genres_mean = pd.DataFrame(genres_mean) # we convert the groupby object to Dataframe for ease of reference
print(genres_mean)


We notice that cheap apps with multiple genres cost more than cheap apps with a single genre. The opposite happens among the reasonable apps.

For each app, the vote cast by the genres feature is determined by how the price of the app relates to the average of its number of genres: Those that cost less than the average are eligible for price increase.

As an example, suppose "An Amazing App" is a reasonable app that belongs to two genres. If its price is lower than $6.82, then the vote is that its price should increase.

We used a similar strategy for the categories: We computed the average for each category and cast the vote in analogous way.

**Results and Impact**

Upon casting the votes and finding the majority, out of the 733 apps that were analyzed, 401 apps (roughly 55%) were deemed eligible for a price increase.

To estimate the impact that increasing prices could have, we want to find the optimal price and have a sense of how many times each app will be purchased.

It wasn't the goal of this prototype to determine the optimal price, so as a quick proxy, we used the average price for each app's affordability. As for the amount of times each app will be purchased, we don't have access to the data, so we tried to use the number of installations instead.

This proxy is not very accurate and realistic, but gives a feel of the potential of a price increase in low costing apps.

In any case, given that roughly 60% of apps seem to be undervalued, our recommendation is that we move forward with this project. 

**Recommendations/ Next Steps**

Some possible next steps are:

- Including unused data in the analysis.
- Get more data, such as:
 - More features — like the number of times each app was purchased.
 - A greater volume of data (we were working with just a little over 700 data points).
 - Select a sample of the eligible apps, increase their price and monitor the behavior of the purchases.
 - Determine optimal prices for each app.